# Проект: Исследование надёжности заёмщиков

***

## Исходная информация

**Заказчик**: кредитный отдел банка.  

**Цель проекта**: определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.  

**Входные данные от банка** : статистика о платёжеспособности клиентов.  

**Дополнительная информация**: результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

***

## 1 Изучение файла с данными

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20227,0,-1347.492896,55,среднее,1,женат / замужем,0,F,сотрудник,0,149480.505901,профильное образование
19482,1,-2350.438414,44,высшее,0,женат / замужем,0,F,компаньон,0,280944.222087,на покупку подержанного автомобиля
19903,0,401215.459785,61,среднее,1,женат / замужем,0,F,пенсионер,0,113380.190187,сделка с подержанным автомобилем
8796,0,-1612.690687,32,среднее,1,женат / замужем,0,F,сотрудник,0,156421.727914,ремонт жилью
19624,0,371707.119269,58,среднее,1,вдовец / вдова,2,F,пенсионер,0,134388.026397,получение высшего образования


In [3]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Выводы:**:
* Набор данных состоит из `21525` наблюдений;
* Столбцы `days_employed` и `total_income` содержат пустые значения, необходимо их проанализировать и при необходимости от них избавиться;
* Столбец `days_employed` содержит отрицательные значения;
* Нужно проверить, какие значения принимают столбцы для поиска ошибок;
* Названия столбцов записаны в нижнем регистре, без лишних пробелов, на одном языке
* Тип данных в столбцах `days_employed` и `total_income` нужно заменить на целочисленный.  

***

## 2 Предобработка данных

### 2.1 Анализ значений, которые могут принимать столбцы в наборе данных

Изучим значения, которые может принимать столбец `children`.

In [5]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

**Выводы**:  

В столбце `children` есть ошибочное/случайное значение `-1` в количестве 47 штук (0,2% от выборки) и выброс `20` в количестве 76 штук (0,3%): 
* Можно предположить, что знак `-` был проставлен случайно, а количество детей у таких клиентов равно 1. Поэтому следует заменить все отрицательные значение в столбце на положительные.  
* Наличие у семьи 20-ти детей маловероятно, скорее всего это ошибочное значение. Необходимо заменить его на среднее (можно попробовать посчитать среднее по каждому из значений столбца `family_status`).  

Изучим значения, которые может принимать столбец `dob_years`.

In [6]:
df['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [7]:
print('Количество строк с нулевым возрастом:', len(df[df['dob_years'] == 0]))
print('Доля строк с нулевым возрастом во всем датасете:', len(df[df['dob_years'] == 0]) / len(df))

Количество строк с нулевым возрастом: 101
Доля строк с нулевым возрастом во всем датасете: 0.004692218350754936


**Выводы**:

В столбце `dob_years` есть ошибочное/случайное значение `0` в количестве 101 штука (0,4% от выборки). Необходимо заменить его на среднее (можно попробовать посчитать среднее по каждому из значений столбца `income_type`).  

Изучим значения, которые может принимать столбец `education`.

In [8]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

**Выводы**:

Необходимо привести значения в столбце `education` к единому варианту написания.  

Изучим значения, которые может принимать столбец `family_status`.

In [9]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

**Выводы**:

В столбце `family_status` ошибок нет.  

Изучим значения, которые может принимать столбец `gender`.

In [10]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

**Выводы**:

В столбце `gender` есть ошибочное/случайное значение `XNA` в количестве 1 штука.  
Его можно заменить на значение большинства (`F`) без искажения результата.

Изучим значения, которые может принимать столбец `income_type`.

In [11]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

**Выводы**:

В столбце `income_type` ошибок нет.

Изучим значения, которые может принимать столбец `purpose`.

In [12]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Выводы**:

В столбце `purpose` есть проблема в записи вариантов ответа (один и тот же вариант ответа записан разными словами).

### 2.2 Обработка пропущенных значений

Предположим, что пропущенные значения в столбцах `days_employed` принадлежат только безработным людям.  
Проверим эту гипотезу.

In [13]:
df[df['days_employed'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [14]:
df[df['total_income'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

**Выводы**:
* Пустые значения в столбцах `days_employed` и `total_income` не принадлежат безработным людям.
* В наблюдениях с непроставленным стажем работы не заполнен и уровень дохода (полученные результаты идентичны).   
* Можно предположить, что данные клиенты просто не захотели указывать свой стаж работы и уровень дохода.
* Так как столбец `days_employed` нам не понадобится для ответа на поставленный заказчиком вопрос, заменим пустые значения в нем на 0.
* Столбец `total_income` будем использовать далее для нахождения зависимости между уровнем дохода и возвратом кредита в срок. Поэтому пропуски в столбце заменим на среднее по категории `income_type`.

Заменим пустые значения в столбцах `days_employed` и `total_income` согласно выводу выше.

In [15]:
df['days_employed'] = df['days_employed'].fillna(0)

mean_income = df.groupby('income_type').agg({'total_income': 'mean'})
for inc_type in mean_income.index:
    inc_med = mean_income.loc[inc_type, 'total_income']
    df.loc[df['income_type'] == inc_type, 'total_income'] = df.loc[df['income_type'] == inc_type, 'total_income'].fillna(inc_med)
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### 2.3 Обработка ошибочных значений

Заменим значение 20 в столбце `children` на среднее по категориям `family_status`.

In [16]:
mean_children = df.groupby('family_status').agg({'children': 'mean'}).round()

for family_status in mean_children.index:
    children_mean = mean_children.loc[family_status, 'children']
    df.loc[df['family_status'] == family_status, 'children'] = df.loc[df['family_status'] == family_status, 'children'].replace(20, children_mean)

df['children'] = df['children'].astype('int')

Заменим отрицательные значения в столбцах `days_employed` и `children` на положительные.

In [17]:
df['days_employed'] = df['days_employed'].abs()
df['children'] = df['children'].abs()

Заменим значение 0 в столбце `dob_years` на среднее по категориям `income_type`.

In [18]:
mean_dob_years = df.groupby('income_type').agg({'dob_years': 'mean'}).round()

for income_type in mean_dob_years.index:
    dob_years_mean = mean_dob_years.loc[income_type, 'dob_years']
    df.loc[df['income_type'] == income_type, 'dob_years'] = df.loc[df['income_type'] == income_type, 'dob_years'].replace(0,dob_years_mean)

df['dob_years'] = df['dob_years'].astype('int')

Заменим ошибочное значение `XNA` в столбце `gender` на значение большинства.

In [19]:
df['gender'] = df['gender'].replace('XNA', 'F')

### 2.4 Замена типа данных

Заменим тип данных в столбцах `days_employed` и `total_income` на целочисленный.

In [20]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

### 2.5 Обработка дубликатов

Преобразуем столбец `education` для удаления дубликатов в нем.  
Для этого сменим регистр в столбце на нижний.

In [21]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Посмотрим, есть ли в таблице дублирующиеся строки.

In [22]:
df.duplicated().sum()

71

Так как у нас нет никакого идентификатора пользователя (+ данные в столбцах `total_income` и `days_employed` с малой долей вероятности совпадут у разных людей), предположим, что повторяющиеся строки - это действительно один и тот же человек и удалим дубликаты из данных.  
Причиной появления дублей скорее всего мог быть технический сбой.

In [23]:
df = df.drop_duplicates().reset_index(drop = True)

### 2.6 Лемматизация

Сформируем таблицу-справочник с имеющимися вариантами целей получения кредита.

In [24]:
purpose = df['purpose'].unique()
purpose_table = pd.DataFrame(data = purpose, columns = ['purpose'])
purpose_table

,purpose
0,покупка жилья
1,приобретение автомобиля
2,дополнительное образование
3,сыграть свадьбу
4,операции с жильем
5,образование
6,на проведение свадьбы
7,покупка жилья для семьи
8,покупка недвижимости
9,покупка коммерческой недвижимости


Лемматизируем список имеющихся целей получения кредита и сформируем столбец с уникальными категориями.

In [25]:
m = Mystem() 
purpose_table['lemmas'] = purpose_table['purpose'].apply(m.lemmatize)

def purpose_category(lemmas):
    for lemma in lemmas:
        if lemma in ('жилье', 'недвижимость'):
            return 'Займ на операции с недвижимостью'
        if lemma == 'автомобиль':
            return 'Займ на покупку автомобиля'
        if lemma == 'образование':
            return 'Займ на получение образования'
        if lemma == 'свадьба':
            return 'Займ на проведение свадьбы'

purpose_table['purpose_category'] = purpose_table['lemmas'].apply(purpose_category)
purpose_table = purpose_table.loc[:,['purpose','purpose_category']]
purpose_table

,purpose,purpose_category
0,покупка жилья,Займ на операции с недвижимостью
1,приобретение автомобиля,Займ на покупку автомобиля
2,дополнительное образование,Займ на получение образования
3,сыграть свадьбу,Займ на проведение свадьбы
4,операции с жильем,Займ на операции с недвижимостью
5,образование,Займ на получение образования
6,на проведение свадьбы,Займ на проведение свадьбы
7,покупка жилья для семьи,Займ на операции с недвижимостью
8,покупка недвижимости,Займ на операции с недвижимостью
9,покупка коммерческой недвижимости,Займ на операции с недвижимостью


Перенесем в исходную таблицу названия полученных категорий.

In [26]:
df = df.merge(purpose_table, on = 'purpose', how = 'left')

### 2.7 Категоризация данных 

Для удобства категоризируем данные в столбцах `children`, `dob_years` и `total_income`.  
Начнем со столбца `children`, который может принимать значения от `0` до `5`.  
Поделим всех клиентов на 3 группы:
* клиенты, не имеющие детей;
* клиенты с 1-2 детьми;
* многодетные семьи.

In [27]:
def children_group(children):
    if children == 0:
        return 'нет детей'
    if 1 <= children <= 2:
        return '1-2 ребенка'
    return 'многодетная семья'
df['children_group'] = df['children'].apply(children_group)

Столбец `dob_years`, который принимает значения от `19` до `75`, разделим также на 3 группы:
* 18-35 (молодежь);
* 36-65 (взрослые люди);
* старше 65 (пенсионеры).

Возраст младше 18 не включаем в категоризацию, так как кредиты детям и подросткам не выдают.

In [28]:
def dob_years_group(dob_years):
    if 18 <= dob_years < 36:
        return '18-35 (молодежь)'
    if 36 <= dob_years < 66:
        return '36-65 (взрослые люди)'
    return 'старше 65 (пенсионеры)'
df['dob_years_group'] = df['dob_years'].apply(dob_years_group)

Посмотрим основную информацию о столбце `total_income`.

In [29]:
df['total_income'].describe().astype('int')

count      21454
mean      167431
std        98060
min        20667
25%       107623
50%       151887
75%       202417
max      2265604
Name: total_income, dtype: int64

Разделим клиентов по столбцу `total_income` на 4 группы согласно персентилям:
* низкий доход (менее 110 тыс.);
* средний доход (от 110 тыс. до 155 тыс.);
* высокий доход (от 155 тыс. до 205 тыс.);
* сверхдоход (более 205 тыс.).

In [30]:
def total_income_group(total_income):
    if total_income <= 110000:
        return 'низкий доход'
    if 110001 <= total_income <= 155000:
        return 'средний доход'
    if 155001 <= total_income <= 205000:
        return 'высокий доход'
    return 'сверхдоход'
df['total_income_group'] = df['total_income'].apply(total_income_group)

***

## 3 Анализ зависимостей и формулировка выводов  

### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Найдем процент должников среди различных категорий клиентов по количеству детей.

In [31]:
children_group_table = df.groupby('children_group').agg({'debt': ['count', 'sum']})
children_group_table['share_of_debtors'] = children_group_table['debt']['sum'] / children_group_table['debt']['count']
children_group_table['share_of_debtors'] = children_group_table['share_of_debtors'].map('{:.2%}'.format)
children_group_table.reset_index()

children_group   debt       share_of_debtors
                      count   sum                 
0        1-2 ребенка   6968   645            9.26%
1  многодетная семья    380    31            8.16%
2          нет детей  14106  1065            7.55%

**Выводы:**
* Выделяется группа клиентов с количеством детей, равным 1-2: по ним наибольший % невозврата кредита среди остальных групп клиентов.  
* Наименьший % должников наблюдается в группе клиентов, у которых детей нет.

### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Найдем процент должников среди различных категорий клиентов по семейному положению.

In [32]:
family_status_table = df.groupby('family_status').agg({'debt': ['count', 'sum']})
family_status_table['share_of_debtors'] = family_status_table['debt']['sum'] / family_status_table['debt']['count']
family_status_table['share_of_debtors'] = family_status_table['share_of_debtors'].map('{:.2%}'.format)
family_status_table.reset_index()

family_status   debt      share_of_debtors
                          count  sum                 
0  Не женат / не замужем   2810  274            9.75%
1              в разводе   1195   85            7.11%
2         вдовец / вдова    959   63            6.57%
3       гражданский брак   4151  388            9.35%
4        женат / замужем  12339  931            7.55%

**Выводы:**
* Наибольший % должников по кредиту наблюдается в группах с семейным положением `Не женат / не замужем` и `гражданский брак`.  
* В группах с семейным положением `женат / замужем` и `в разводе` примерно одинаковый % должников.  
* Наименьший % должников имеет группа `вдовец / вдова`. 

### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок? 

Найдем процент должников среди различных категорий клиентов по уровню дохода.

In [33]:
total_income_group_table = df.groupby('total_income_group').agg({'debt': ['count', 'sum']})
total_income_group_table['share_of_debtors'] = total_income_group_table['debt']['sum'] / total_income_group_table['debt']['count']
total_income_group_table['share_of_debtors'] = total_income_group_table['share_of_debtors'].map('{:.2%}'.format)
total_income_group_table.reset_index()

total_income_group  debt      share_of_debtors
                     count  sum                 
0      высокий доход  5698  490            8.60%
1       низкий доход  5642  455            8.06%
2         сверхдоход  4735  330            6.97%
3      средний доход  5379  466            8.66%

**Выводы:**
* Клиенты с `низким`, `средним` и `высоким` доходом имеют примерно одинаковую долю должников в своих группах.  
* Немного выделяется группа клиентов со `сверхдоходом`: % невозврата кредита в срок по ним ниже, чем по остальным группам клиентов. 

### 3.4 Как разные цели кредита влияют на его возврат в срок?

Найдем процент должников среди различных категорий клиентов по цели кредита.

In [34]:
purpose_category_table = df.groupby('purpose_category').agg({'debt': ['count', 'sum']})
purpose_category_table['share_of_debtors'] = purpose_category_table['debt']['sum'] / purpose_category_table['debt']['count']
purpose_category_table['share_of_debtors'] = purpose_category_table['share_of_debtors'].map('{:.2%}'.format)
purpose_category_table.reset_index()

purpose_category   debt      share_of_debtors
                                     count  sum                 
0  Займ на операции с недвижимостью  10811  782            7.23%
1        Займ на покупку автомобиля   4306  403            9.36%
2     Займ на получение образования   4013  370            9.22%
3        Займ на проведение свадьбы   2324  186            8.00%

**Выводы:**
* Наибольший % невозврата кредита наблюдается по кредитам на `покупку автомобиля` и на `получение образования`.  
* Самый низкий % должников находится в группе кредит на `операции с недвижимостью`.

***

## 4 Общий вывод по проекту 

После анализа предоставленных заказчиком данных можно сделать следующие выводы об имеющихся зависимостях:


1. **Зависимость между количеством детей и возвратом кредита в срок:**
    * Клиенты, не имеющие детей, с большой долей вероятности вернут кредит в срок;  
    * Для клиентов с 1-2 детьми риск невозврата кредита в срок увеличивается на 1.7 п.п.;  
    * Вероятность невозврата кредита у клиентов, имеющих многодетные семьи, увеличивается на 0.6 п.п. по сравнению с клиентами без детей.
    
    
2. **Зависимость между семейным положением и возвратом кредита в срок**:
    * Наибольший и примерно одинаковый % должников по кредиту наблюдается в группах с семейным положением `Не женат / не замужем` и `гражданский брак`.  
    * В группах с семейным положением `женат / замужем` и `в разводе` также примерно одинаковый % должников.  
    * Так как значения доли должников в диаметрально противоположных группах клиентов схожи, можно сделать вывод, что зависимости между семейным положением клиента и невозвратом кредита в срок нет.
    
    
3. **Зависимость между уровнем дохода и возвратом кредита в срок:**
    * Клиенты с `низким`, `средним` и `высоким` доходом имеют примерно одинаковую долю должников в своих группах.  
    * % невозврата кредита в срок по группе клиентов со `сверхдоходом` ниже, чем по остальным группам клиентов. Это можно объяснить тем, что у данной категории клиентов проблем с деньгами и их отсутствием меньше, чем у остальных групп.
    
    
4. **Зависимость между целью кредита и возвратом кредита в срок:**
    * Кредиты, взятые с целью `покупки автомобиля` или `получения образования` с большей вероятностью не будут возвращены в срок.  
    * Самый низкий % должников находится в группе кредит `на операции с недвижимостью`. Возможно, это связано с тем, что срок возврата по таким кредитам обычно очень большой.

***

## 5 Рекомендации

Скорее всего данные о клиентах были получены путем анкетирования.  
Если это действительно так, то предлагаю следующие рекомендации:
* Для упрощения обработки данных рекомендуется заполнять информацию в ячейках `education`, `gender`, `purpose` готовыми вариантами ответа, чтобы унифицировать значения и избежать ошибочного заполнения данных.
* Для избежания ошибок в столбцах `children` и `dob_years` необходимо на входе проверять введенные клиентом значения на их ошибочность (детей не может быть -1, возраст не может равняться нулю).
* Значения в столбце `days_employed` требуют доработки, в данном виде они непригодны к анализу.